In [1]:
import pandas_datareader.data as web
from datetime import datetime
import numpy as np
import pandas as pd
from multiprocessing import Pool
import tqdm
import gc

# Local 
import GetStockData
from StockAnalysis import Analysis

#### Load List of Stock Symbol

In [2]:
GetStockData.update_tickers_list()
nasdaq = GetStockData.get_nasdaq_tickers()
nasdaq.shape

(5884, 2)

In [3]:
# Get today's date
t = datetime.now().strftime('%m%d%Y')

In [4]:
# Create an empty list to store data 
bbands_outliers=[]

In [5]:
# Define a function to find out all Bollinger Band Pattern, using with multiprocessing 

def All_BBands_Outliers(symbol, start = datetime(2020,3,1), end = datetime.now(), last_n_days = 3):
    return Analysis(symbol, start, end).BBand_Outliers(last_n_days = last_n_days)

In [6]:
%%time

n_cpu = 4

with Pool(processes=n_cpu) as p:
    max_ = len(nasdaq.Symbol)
    with tqdm.tqdm_notebook(total=max_) as pbar: # May need to change the tqdm_notebook later
        for i, r in enumerate(p.imap(All_BBands_Outliers, nasdaq.Symbol)):
            bbands_outliers.append(r)
            pbar.update(1)


CPU times: user 7.48 s, sys: 2.42 s, total: 9.91 s
Wall time: 52min 13s


In [7]:
len(bbands_outliers)

63

In [8]:
bbands_outliers = [b for b in bbands_outliers if b != None]
results = pd.DataFrame(bbands_outliers, columns=['Symbol', 'BBIndicator'])
results = results[(~results['BBIndicator'].isnull())]

In [9]:
results.head()

,Symbol,BBIndicator
2,AACG,On Lower Band
5,AAME,On Lower Band
7,AAOI,On Upper Band
13,AAWW,On Upper Band
21,ABEV,On Lower Band


In [10]:
path = r'~/stock_analysis/Bollinger Band Analysis/'
filename = "Bollinger_Band_Indicator" + t + ".csv"
results.to_csv(path+filename, index=False)